# Snapshot & Solver Tuning

Demonstrates the `snapshot()` context manager for safe what-if analysis
and the solver option descriptors for tuning Newton-Raphson convergence.
The notebook covers saving and restoring state, comparing solver
configurations, and stress-testing the system under modified conditions.

Import the case and instantiate the `PowerWorld`.

```python
import numpy as np
from esapp import PowerWorld
from esapp.components import *

pw = PowerWorld(case_path)
```

In [ ]:
import numpy as np
from esapp import PowerWorld
from esapp.components import *
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

In [ ]:
import sys; sys.path.insert(0, '..')
from plot_helpers import plot_snapshot_comparison, plot_voltage_profile

## 1. Baseline State

Solve the base case and record the initial voltage profile.

In [ ]:
pw.pflow()
v_base = pw.voltage(complex=False, pu=True)[0]
print(f"Base voltage range: {v_base.min():.4f} - {v_base.max():.4f} pu")
print(f"Buses below 0.95 pu: {(v_base < 0.95).sum()}")

## 2. What-If with Snapshot

The `snapshot()` context manager saves the full case state on entry
and restores it on exit. This lets you make arbitrary modifications,
solve, and inspect results without corrupting the base case.

In [ ]:
# Increase all loads by 20% inside a snapshot
with pw.snapshot():
    loads = pw.loads()
    loads['LoadMW'] *= 1.20
    pw[Load, 'LoadMW'] = loads[['BusNum', 'LoadID', 'LoadMW']]
    pw.pflow()

    v_stressed = pw.voltage(complex=False, pu=True)[0]
    print(f"Stressed voltage range: {v_stressed.min():.4f} - {v_stressed.max():.4f} pu")
    print(f"Buses below 0.95 pu: {(v_stressed < 0.95).sum()}")
    overloads = pw.overloads()
    print(f"Overloaded branches: {len(overloads)}")

# Verify restoration
v_restored = pw.voltage(complex=False, pu=True)[0]
print(f"\nAfter snapshot exit, max |dV| from base: {np.abs(v_restored - v_base).max():.6f}")

In [ ]:
plot_snapshot_comparison(v_base.values, v_stressed.values)

## 3. Generator Trip Study

Use `snapshot()` to study the impact of tripping the largest generator.

In [ ]:
gens = pw.gens()
online = gens[gens['GenStatus'] == 'Closed']
largest_gen_idx = online['GenMW'].idxmax()
gen_bus = int(online.loc[largest_gen_idx, 'BusNum'])
gen_mw = online.loc[largest_gen_idx, 'GenMW']
print(f"Largest online generator: Bus {gen_bus}, {gen_mw:.1f} MW")

with pw.snapshot():
    # Trip the generator
    trip_df = online.loc[[largest_gen_idx], ['BusNum', 'GenID']].copy()
    trip_df['GenStatus'] = 'Open'
    pw[Gen] = trip_df
    pw.pflow()

    v_tripped = pw.voltage(complex=False, pu=True)[0]
    violations = pw.violations(v_min=0.90, v_max=1.10)
    n_low = violations['Low'].dropna().shape[0]
    n_high = violations['High'].dropna().shape[0]
    print(f"After trip: {n_low} low-voltage, {n_high} high-voltage violations")
    print(f"Voltage drop at gen bus: {v_base.iloc[0]:.4f} → {v_tripped.iloc[0]:.4f} pu")

print("State restored after snapshot.")

## 4. Solver Option Descriptors

PowerWorld solver settings are exposed as Python descriptors on the
`PowerWorld` instance. Reading an attribute queries PowerWorld; setting
it pushes the value immediately.

In [ ]:
# Read current settings
print(f"Max iterations:    {pw.max_iterations}")
print(f"Convergence tol:   {pw.convergence_tol}")
print(f"Flat start:        {pw.flat_start}")
print(f"Check taps:        {pw.check_taps}")
print(f"Check shunts:      {pw.check_shunts}")
print(f"Disable opt mult:  {pw.disable_opt_mult}")

## 5. Flat Start Convergence

Compare solving from the current state vs. a flat start (all voltages
at 1.0 pu, 0 degrees).

In [ ]:
# Normal solve (warm start)
with pw.snapshot():
    pw.pflow()
    v_warm = pw.voltage(complex=False, pu=True)[0]
    P_mm, Q_mm = pw.mismatch()
    print(f"Warm start: max |P mismatch| = {P_mm.abs().max():.2e}, "
          f"max |Q mismatch| = {Q_mm.abs().max():.2e}")

# Flat start solve
with pw.snapshot():
    pw.flat_start = True
    pw.pflow()
    pw.flat_start = False
    v_flat = pw.voltage(complex=False, pu=True)[0]
    P_mm, Q_mm = pw.mismatch()
    print(f"Flat start: max |P mismatch| = {P_mm.abs().max():.2e}, "
          f"max |Q mismatch| = {Q_mm.abs().max():.2e}")

print(f"\nMax voltage difference (warm vs flat): {np.abs(v_warm - v_flat).max():.6f} pu")

## 6. Tightening Convergence

Reduce the convergence tolerance and observe the effect on mismatches.

In [ ]:
original_tol = pw.convergence_tol
print(f"Default tolerance: {original_tol}")

with pw.snapshot():
    pw.convergence_tol = 1e-7
    pw.pflow()
    P_tight, Q_tight = pw.mismatch()
    print(f"Tight solve (tol=1e-7): max |P| = {P_tight.abs().max():.2e}, "
          f"max |Q| = {Q_tight.abs().max():.2e}")

# Restore original tolerance
pw.convergence_tol = original_tol
print(f"Tolerance restored to: {pw.convergence_tol}")

## 7. Comparing Voltage Impact Across Load Levels

Use `snapshot()` in a loop to sweep load scaling factors and collect
the minimum voltage at each level.

In [ ]:
scales = np.arange(0.8, 1.35, 0.05)
v_mins = []
v_maxs = []

for scale in scales:
    with pw.snapshot():
        loads = pw.loads()
        loads['LoadMW'] *= scale
        pw[Load, 'LoadMW'] = loads[['BusNum', 'LoadID', 'LoadMW']]
        pw.pflow()
        v = pw.voltage(complex=False, pu=True)[0]
        v_mins.append(v.min())
        v_maxs.append(v.max())

print("Load Scale | V_min   | V_max")
print("-" * 33)
for sc, vn, vx in zip(scales, v_mins, v_maxs):
    flag = ' !!!' if vn < 0.90 else ''
    print(f"  {sc:.2f}     | {vn:.4f} | {vx:.4f}{flag}")

In [ ]:
plot_voltage_profile(v_base)